# Tratamento da base de dados

In [ ]:
import pandas as pd
import numpy as np
base_census = pd.read_csv('census.csv')
colunas = []
colunas.append('income')
for i in range(len(base_census.columns[:-1])):
    colunas.append(base_census.columns[i])
base_census = base_census[colunas]
base_census

In [ ]:
def income(text):
    if text == ' >50K':
        return 1.0
    else:
        return 0.0

In [ ]:
base_census['income'] = base_census['income'].apply(income)

In [ ]:
base_census

In [ ]:
base_census = pd.get_dummies(base_census,prefix=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'inative-country'], 
                            columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'inative-country'])

In [ ]:
base_treinamento = base_census.iloc[0:22792,:]
base_treinamento.shape

In [ ]:
22792 + 9769

In [ ]:
X_teste = base_teste.iloc[:,1:len(base_census)].values
X_teste

In [ ]:
y_teste = base_teste.iloc[:, 0].values
y_teste

In [ ]:
base_treinamento.to_csv('census_train_xgboost.csv', header = False, index = False)
base_teste.to_csv('census_test_xgboost.csv', header = False, index = False)

# Configurações SageMaker

In [ ]:
import sagemaker
import boto3
from sagemaker import Session
import sagemaker.amazon.common as smac # sagemaker commom library
import io
import os

In [ ]:
session = sagemaker.Session()
bucket = 'cursoawssagemaker'
subpasta_modelo = 'modelos/census/xgboost'
subpasta_dataset = 'datasets/census'
key_train = 'census-train-data-xgboost'
key_test = 'census-test-data-xgboost'
role = sagemaker.get_execution_role()
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, subpasta_dataset, key_train)
s3_test_data = 's3://{}/{}/test/{}'.format(bucket, subpasta_dataset, key_test)
output_location = 's3://{}/{}/output'.format(bucket, subpasta_modelo)
print('Role: {}'.format(role))
print('Localização da base de treinamento: {}'.format(s3_train_data))
print('Localização da base de teste: {}'.format(s3_test_data))
print('Modelo final será salvo em: {}'.format(output_location))

In [ ]:
with open('census_train_xgboost.csv', 'rb') as f:
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'train', key_train)).upload_fileobj(f)

In [ ]:
with open('census_test_xgboost.csv', 'rb') as f:
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'test', key_test)).upload_fileobj(f)